In [ ]:
def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row['Sentence'])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_.loc[len(df_)] = {
            "index": row['index'],
            "Class": row['Class'],
            "Sentence": " ".join(filtered_sent)
        }
    return data

# If this is the primary file that is executed (ie not an import of another file)
if __name__ == "__main__":
    # get data, pre-process and split
    training_xlarge = pd.read_csv("train.csv", delimiter=',', header=None).sample(200000)            # choose a sample of size 200000
    #test_xlarge = pd.read_csv("test.csv", delimiter=',', header=None)
    training_xlarge.columns = [ 'Class', 'Title', 'Sentence']
    training_xlarge['index'] = training_xlarge.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    training_xlarge = preprocess_pandas(training_xlarge, columns)                             # pre-process
    
    training_data_xl, validation_data_xl, training_labels_xl, validation_labels_xl = train_test_split( # split the data into training, validation, and test splits
        training_xlarge['Sentence'].values.astype('U'),
        training_xlarge['Class'].values.astype('int32'),
        test_size=0.10,
        random_state=0,
        shuffle=True
    )
    training_labels_xl -= 1                       # mapping from {1, 2} to {0, 1}
    validation_labels_xl -= 1
